<a href="https://colab.research.google.com/github/J0SAL/pyspark-play/blob/main/spark_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topics Covered
- Mlib intro with custom dataset

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=30c10f7da1d3cd96fdece9d492171dc5ed7822e5f7d31bb6f58f46700deb73f4
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
# Creating Spark Session
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [3]:
spark

In [18]:
# read dataset - way1
filepath = '/content/drive/MyDrive/Colab Notebooks/spark/test1.csv'
training=spark.read.option('header','true').csv(filepath,inferSchema=True)

In [21]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [20]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [22]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
'''
steps
1. group up the training features
2. these will be treated as new features
3. these new features will be used as independent features

age, experience -> [age, experience] -> new features -> independent features
'''

In [24]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [26]:
output=featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [27]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [29]:
finalized_data=output.select("Independent Features","Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [38]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.55,0.45])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [40]:
train_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [23.0,2.0]| 18000|
|          [29.0,4.0]| 20000|
|          [30.0,8.0]| 25000|
+--------------------+------+



In [41]:
### Coefficients
regressor.coefficients

DenseVector([-90.9091, 1272.7273])

In [42]:
### Intercepts
regressor.intercept

17545.454545455603

In [43]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [44]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000| 16909.09090909103|
|          [24.0,3.0]| 20000| 19181.81818181826|
|         [31.0,10.0]| 30000|27454.545454545536|
+--------------------+------+------------------+



In [45]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1757.5757575757452, 3597796.143250664)